In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

In [2]:
IPL_ESPN_Mapping = './../MappingFiles/IPL_ESPN_Mapping.csv'

In [3]:
df = pd.read_csv('./../Inputs/IPL/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv')

In [4]:
df_unique_batsman = pd.DataFrame(df['batsman'].unique(), columns=['Players'])
df_unique_non_striker = pd.DataFrame(df['non_striker'].unique(), columns=['Players'])
df_unique_bowler = pd.DataFrame(df['bowler'].unique(), columns=['Players'])
df_unique_fielder = pd.DataFrame(df['fielder'].unique(), columns=['Players'])
df_unique_player_dismissed = pd.DataFrame(df['player_dismissed'].unique(), columns=['Players'])
df_unique_players = pd.concat([df_unique_batsman, df_unique_non_striker, df_unique_bowler, df_unique_fielder, df_unique_player_dismissed])
df_unique_players = pd.DataFrame(df_unique_players['Players'].unique(), columns=['Players'])
df_unique_players

,Players
0,RT Ponting
1,BB McCullum
2,DJ Hussey
3,Mohammad Hafeez
4,SC Ganguly
...,...
990,PK Garg (sub)
991,SP Goswami (sub)
992,Lalit Yadav (sub)
993,P Negi (sub)


In [5]:
# method to get the full name of player from the espn crickinfo website

def getFullName(name):
    temp = name + " cricket player"
    googleLink = 'https://www.google.com/search'
    response = requests.get(googleLink, params={'q': temp}).text
    # print(response)
    soup = BeautifulSoup(response, features='html.parser').find_all('a')
    # print(soup)
    # name = soup.find_all("h3", class_ = "name link-cta")
    try:
        # print(soup)
        # allLinks = [link.get('href') for link in soup]
        # all_links = []
        # print(soup)
        for url_link in soup:
            temp_url = url_link.get("href")
            # print(temp_url)
            if "espncricinfo" in temp_url and "img" not in temp_url and "image" not in temp_url:
                temp_url = temp_url.replace("/url?q=","").split('&')[0]
                # print(temp_url)
                break
        responsePlayer = requests.get(temp_url).text
        soup = BeautifulSoup(responsePlayer, features='html.parser')
        # print(soup)
        fullName = soup.find_all("span", class_ = "ds-text-title-s ds-font-bold ds-text-typo")[0].find("p").text
        return fullName
    except:
        return "NA"

In [6]:
# replace the short name with full Name
dict = {}
for player in df_unique_players['Players']:
    player = str(player)
    if "," not in player and "(sub)" not in player:
        try:
            dict[player] = getFullName(str(player))
        except:
            print("NA")
    # df_processed.replace(player, getFullName(player), inplace=True)
# print(getFullName("RT Ponting"))
# df_processed.replace('RT Ponting', 'Rickey Ponting')
# df_processed
print(dict)

{'RT Ponting': 'Ricky Thomas Ponting', 'BB McCullum': 'Brendon Barrie McCullum', 'DJ Hussey': 'David John Hussey', 'Mohammad Hafeez': 'Mohammad Hafeez', 'SC Ganguly': 'Sourav Chandidas Ganguly', 'CL White': 'Cameron Leon White', 'MV Boucher': 'Mark Verdon Boucher', 'B Akhil': 'Balachandra Akhil', 'AA Noffke': 'Ashley Allan Noffke', 'P Kumar': 'Praveenkumar Sakat Singh', 'Z Khan': 'Zaheer Khan', 'SB Joshi': 'Sunil Bandacharya Joshi', 'W Jaffer': 'Wasim Jaffer', 'JH Kallis': 'Jacques Henry Kallis', 'R Dravid': 'Rahul Sharad Dravid', 'V Kohli': 'Virat Kohli', 'K Goel': 'Karan Goel', 'MEK Hussey': 'Michael Edward Killeen Hussey', 'JR Hopes': 'James Redfern Hopes', 'SK Raina': 'Suresh Kumar Raina', 'JDP Oram': 'Jacob David Philip Oram', 'S Badrinath': 'Subramaniam Badrinath', 'PA Patel': 'Parthiv Ajay Patel', 'ML Hayden': 'Matthew Lawrence Hayden', 'MS Dhoni': 'Mahendra Singh Dhoni', 'KC Sangakkara': 'Kumar Chokshanada Sangakkara', 'Yuvraj Singh': 'Yuvraj Singh', 'SM Katich': 'Simon Mathew 

In [7]:
condition = os.path.exists(IPL_ESPN_Mapping)
condition

False

In [9]:
if not(condition):
    os.mkdir('./../MappingFiles/')
    pd.DataFrame.from_dict(dict, orient='index').to_csv(IPL_ESPN_Mapping)